In [1]:
import emoji
import pandas as pd
import numpy as np
import faiss
import pickle
import re
from pymorphy3 import MorphAnalyzer
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\VORANDPAV_BIG_SPB\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
QUESTIONS_PATH = "../Data/Common/questions_clean.csv"
WEBSITES_PATH = "../Data/Common/websites_updated.csv"
SUBMISSION_PATH = "../Data/Common/sample_submission.csv"

OUTPUT_SUBMISSION_FILE = "submission.csv"

EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-large"
RERANKING_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-12-v2"

INDEX_PATH = "faiss_index.bin"
METADATA_PATH = "metadata.pkl"

MAX_TOKENS = 512
MIN_TOKENS = 64
SIM_THRESHOLD = 0.8
OVERLAP_SENTENCES = 2

TOP_K = 5

In [3]:
morph = MorphAnalyzer()


def clean_text(text: str) -> str:
    """
    Очищает текст от HTML-тегов, лишних символов и приводит к нижнему регистру.
    """
    # Удаляем HTML-теги
    text = re.sub(r'<[^>]+>', ' ', text)
    # Удаляем URL-адреса
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Удаляем знаки препинания и цифры, оставляем только русские буквы и пробелы
    text = re.sub(r'[^а-яА-ЯёЁ\s]', ' ', text)
    # Заменяем множественные пробелы на один
    text = re.sub(r'\s+', ' ', text).strip()
    # Приводим к нижнему регистру
    text = text.lower()
    return text


def lemmatize_text(text: str) -> str:
    """
    Лемматизирует текст (приводит слова к начальной форме).
    """
    words = text.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return " ".join(lemmatized_words)


def preprocess_query(query: str) -> str:
    """
    Полный цикл предобработки для поискового запроса.
    """
    cleaned_query = clean_text(query)
    lemmatized_query = lemmatize_text(cleaned_query)
    return lemmatized_query


def preprocess_document(text: str) -> str:
    """
    Полный цикл предобработки для текста документа.
    В данном бейзлайне он такой же, как для запроса.
    """
    cleaned_doc = clean_text(text)
    # Для документов лемматизация может быть опциональной, в зависимости от модели
    # Но для симметричного поиска (когда и запрос, и документ обрабатываются одинаково)
    # лучше ее оставить.
    lemmatized_doc = lemmatize_text(cleaned_doc)
    return lemmatized_doc


def easy_preprocess_text(text):
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


1. Загружает и обрабатывает документы.

2. Разбивает их на чанки.

3. Векторизует чанки.

4. Создает и сохраняет FAISS индекс и метаданные.

In [4]:
# 1. Загрузка данных и моделей

df_websites = pd.read_csv(WEBSITES_PATH)
df_websites['title'] = df_websites['title'].fillna('')
df_websites['text'] = df_websites['text'].fillna('')

In [5]:
df_websites[df_websites['web_id'] == 1705].iloc[0]

web_id                                                 1705
url       https://alfabank.servicecdn.ru/site-upload/f4/...
kind                                               document
title                               dogovor_cbo_1072025.pdf
text      Приложение к Распоряжению АО «АЛЬФА-БАНК» \nот...
Name: 1704, dtype: object

In [6]:
print(f"Загрузка модели эмбеддингов и токенайзера: {EMBEDDING_MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME, device='cuda' if torch.cuda.is_available() else 'cpu')
embeddings_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cuda' if torch.cuda.is_available() else 'cpu')

Загрузка модели эмбеддингов и токенайзера: intfloat/multilingual-e5-large


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /intfloat/multilingual-e5-large/resolve/main/1_Pooling/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B32121DE50>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 567326ea-a533-40e3-8344-fac317f5744f)')' thrown while requesting HEAD https://huggingface.co/intfloat/multilingual-e5-large/resolve/main/1_Pooling/config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /intfloat/multilingual-e5-large/resolve/main/1_Pooling/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B32121CE10>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 5a6e93aa-9cba-4472-ae2e-9634a98e6a52)')' thrown while requesting HEAD https://huggingface

In [7]:
# 2. Создание чанков

def tokenize_len(text):
    return len(tokenizer.encode(text, add_special_tokens=False))


def embed_sentences(sentences):
    # Метод .encode() сам обрабатывает токенизацию, паддинг и запуск на нужном устройстве
    embeddings = embeddings_model.encode(sentences, convert_to_numpy=True, show_progress_bar=False)
    return embeddings


def semantic_chunk_text(text):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 0]
    if len(sentences) <= 1:
        return [text]

    embs = embed_sentences(sentences)
    sims = cosine_similarity(embs[:-1], embs[1:]).diagonal()

    breaks = [i for i, sim in enumerate(sims) if sim < SIM_THRESHOLD]
    chunks, start = [], 0
    for b in breaks + [len(sentences) - 1]:
        chunk = " ".join(sentences[start:b + 1])
        chunks.append(chunk)
        start = b + 1

    merged = []
    buffer = ""
    for c in chunks:
        if tokenize_len(c) < MIN_TOKENS:
            buffer += " " + c
        else:
            if buffer:
                merged.append(buffer.strip())
                buffer = ""
            merged.append(c)
    if buffer:
        merged.append(buffer.strip())

    final_chunks = []
    for i, c in enumerate(merged):
        overlap = " ".join(sentences[max(0, i - OVERLAP_SENTENCES):i])
        full = (overlap + " " + c).strip()
        final_chunks.append(full)
    return final_chunks


def auto_summary(text):
    sents = re.split(r'(?<=[.!?])\s+', text)
    return " ".join(sents[:2]).strip()


def create_chunks_with_headers(df_websites):
    all_chunks = []
    metadata = []
    for _, row in tqdm(df_websites.iterrows(), total=len(df_websites), desc="Semantic chunking"):
        title = str(row['title'])
        text = str(row['text'])
        full_text = f"{title}. {text}"
        sem_chunks = semantic_chunk_text(full_text)
        for ch in sem_chunks:
            header = f"[Документ: {title}] [Краткое содержание: {auto_summary(ch)}] "
            all_chunks.append(header + ch)
            metadata.append({'web_id': row['web_id'], 'title': title})
    return all_chunks, metadata

In [24]:
print("Создание чанков из документов...")
chunks, metadata = create_chunks_with_headers(df_websites[:1000])
print(f"Создано {len(chunks)} чанков.")

Создание чанков из документов...


Semantic chunking: 100%|██████████| 1000/1000 [05:40<00:00,  2.94it/s]

Создано 5979 чанков.


In [25]:
chunks[10]

'[Документ: А-Клуб. Деньги имеют значение] [Краткое содержание: А-Клуб. АО «Альфа-Банк» является участником системы обязательного страхования вкладов.] А-Клуб. АО «Альфа-Банк» является участником системы обязательного страхования вкладов. Информация о процентных ставках по договорам банковского вклада с физическими лицами. Центр раскрытия корпоративной информации. Информация профессионального участника рынка ценных бумаг. Информация о лицах, под контролем либо значительным влиянием которых находится Банк. Ул. Каланчевская, 27, Москва, 107078.'

In [26]:
metadata[10]

{'web_id': 3, 'title': 'А-Клуб. Деньги имеют значение'}

In [27]:
# !nvidia-smi
torch.__version__

'2.9.1+cu130'

In [28]:
# 4. Векторизация чанков
def clean_for_embeddings(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.replace('\xa0', ' ')  # неразрывные пробелы
    text = re.sub(r'\n+', '. ', text)  # переносы -> точки
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'http\S+', '', text)  # убрать URL
    text = re.sub(r'\d+[\s]*(Кб|байт|Mb|GB|Kb|MB|pdf|архив)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(\.|,){2,}', '.', text)  # убрать повторяющиеся точки/запятые
    text = re.sub(r'[«»“”"]', '"', text)
    text = emoji.replace_emoji(text, replace='')  # убрать эмодзи 🚀📚🔥
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()


passage_chunks = ["passage: " + clean_for_embeddings(chunk) for chunk in chunks]
embeddings = embeddings_model.encode(
    passage_chunks,
    batch_size=32,
    convert_to_numpy=True,
    normalize_embeddings=False,
    show_progress_bar=True
)
print(f"Размерность эмбеддингов: {embeddings.shape}")

# Нормализуем эмбеддинги для использования с косинусным расстоянием (IndexFlatIP)
faiss.normalize_L2(embeddings)

d = embeddings.shape[1]  # Размерность векторов
index = faiss.IndexFlatIP(d)  # IndexFlatIP подходит для нормализованных векторов (косинусное сходство)
index.add(embeddings)



Batches: 100%|██████████| 187/187 [03:05<00:00,  1.01it/s]

Размерность эмбеддингов: (5979, 1024)


In [29]:
# 5. Сохранение индекса и метаданных...
faiss.write_index(index, INDEX_PATH)
with open(METADATA_PATH, 'wb') as f:
    pickle.dump(metadata, f)

print("Индексация завершена успешно!")

Индексация завершена успешно!


1. Загружает индекс, модель и данные.

2. Обрабатывает каждый вопрос.

3. Выполняет поиск в индексе.

4. Агрегирует результаты и формирует файл submission.csv.

In [30]:
print("1. Загрузка данных")
index = faiss.read_index(INDEX_PATH)
with open(METADATA_PATH, 'rb') as f:
    metadata = pickle.load(f)

1. Загрузка данных


In [31]:
# embeddings_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cuda' if torch.cuda.is_available() else 'cpu')
df_questions = pd.read_csv(QUESTIONS_PATH)

all_web_lists = []

In [33]:
print("2.0. Поиск релевантных документов для одного вопроса (тест)...")
test_query = "Где узнать бик и счёт"
preprocessed_test_query = clean_for_embeddings(test_query)
test_query_embedding = embeddings_model.encode(["query: " + preprocessed_test_query], convert_to_numpy=True)
faiss.normalize_L2(test_query_embedding)  # Нормализуем вектор запроса
result = index.search(test_query_embedding, TOP_K * 5)
print("Найденные web_id для тестового запроса:")
print("Запрос:", test_query)
print("Предобработанный запрос:", preprocessed_test_query)
print("Вектор запроса:", test_query_embedding[0][:5])
for i in range(TOP_K):
    print(f"Результат {i + 1}: web_id = {metadata[result[1][0][i]]['web_id']}, расстояние = {result[0][0][i]:.4f}")
    print("Текст:", chunks[i])
print(result[0], result[1])

2.0. Поиск релевантных документов для одного вопроса (тест)...
Найденные web_id для тестового запроса:
Запрос: Где узнать бик и счёт
Предобработанный запрос: Где узнать бик и счёт
Вектор запроса: [ 0.02718943 -0.0318291  -0.04523505 -0.05356316  0.00961234]
Результат 1: web_id = 372, расстояние = 0.8329
Текст: [Документ: Альфа-Банк - кредитные и дебетовые карты, кредиты наличными, автокредитование, ипотека и другие банковские услуги физическим и юридическим лицам – Альфа-Банк] [Краткое содержание: Альфа-Банк - кредитные и дебетовые карты, кредиты наличными, автокредитование, ипотека и другие банковские услуги физическим и юридическим лицам – Альфа-Банк. Рассчитайте выгоду
Расчёт калькулятора предварительный.] Альфа-Банк - кредитные и дебетовые карты, кредиты наличными, автокредитование, ипотека и другие банковские услуги физическим и юридическим лицам – Альфа-Банк. Рассчитайте выгоду
Расчёт калькулятора предварительный.
Результат 2: web_id = 368, расстояние = 0.8320
Текст: [Документ: А

In [6]:
print("2. Поиск релевантных документов для каждого вопроса...")
for _, row in tqdm(df_questions.iterrows(), total=df_questions.shape[0], desc="Обработка вопросов"):
    query = easy_preprocess_text(row['query'])
    query_embedding = embeddings_model.encode(["query: " + query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)  # Нормализуем вектор запроса

    # Ищем TOP_K * 5 кандидатов, чтобы иметь запас для выбора уникальных web_id
    distances, indices = index.search(query_embedding, TOP_K * 5)

    found_web_ids = []
    # Собираем уникальные web_id в порядке их нахождения
    for idx in indices[0]:
        if idx != -1:  # FAISS может вернуть -1, если ничего не найдено
            web_id = metadata[idx]['web_id']
            if web_id not in found_web_ids:
                found_web_ids.append(web_id)
            if len(found_web_ids) == TOP_K:
                break

    # Если найдено меньше TOP_K документов, дополняем любыми другими web_id
    # Этого не должно происходить при K*5 поиске, но это защита
    if len(found_web_ids) < TOP_K:
        # Просто добавим первые web_id, которых еще нет в списке
        all_ids = [m['web_id'] for m in metadata]
        for web_id in all_ids:
            if web_id not in found_web_ids:
                found_web_ids.append(web_id)
            if len(found_web_ids) == TOP_K:
                break

    all_web_lists.append(str(found_web_ids))

2. Поиск релевантных документов для каждого вопроса...


Обработка вопросов: 100%|██████████| 6977/6977 [00:55<00:00, 124.63it/s]


In [7]:
print("3. Создание файла для отправки...")
submission_df = pd.DataFrame({
    'q_id': df_questions['q_id'],
    'web_list': all_web_lists
})

submission_df.to_csv(OUTPUT_SUBMISSION_FILE, index=False)
print(f"Файл '{OUTPUT_SUBMISSION_FILE}' успешно создан.")

3. Создание файла для отправки...
Файл 'submission.csv' успешно создан.
